# exploring the Allen Brain Atlas API with Tortilla

Tortilla simply uses Python-familiar structures like attributes and arguments to construct the URL for an API query and return the response. 

### We can start by wrapping the top level of the API url and giving it a friendly name.

In [1]:
import tortilla
allen = tortilla.wrap("http://api.brain-map.org/api/v2/data")

Now, to really explore Allen's API, you have to do more than just traverse a hierarchy of URLs. 

It maintains its own syntax for queries, passed in as the value of the "criteria" parameter. Tortilla can't help us much in constructing this, so we are just going to construct it by assembling the string appropriately.

The full Allen API is beyond the scope of this demo, so we're just going to construct our criteria borrowed from one of Allen's examples (http://www.brain-map.org/api/examples/examples/doc/structures/download_data.py.html)

In [2]:
MOUSE_PRODUCT_ID = 1 # aba
PLANE_ID = 1 # coronal

criteria = ','.join(
    ["[failed$eq'false'][expression$eq'true']",
     "products[id$eq%d]"%MOUSE_PRODUCT_ID,
     "plane_of_section[id$eq%d]" % PLANE_ID,
     ]
    )
print criteria

[failed$eq'false'][expression$eq'true'],products[id$eq1],plane_of_section[id$eq1]


### Once we know our necessary parameters, building the query is easy

We can construct our query with the default parameters passed in as a dictionary and save the wrapper to its own variable.

In [3]:
dataset_parameters = {
    'criteria': criteria,
    'start_row': 0,
    'num_rows': 2000,
}
section_query = allen.SectionDataSet('query.json',params=dataset_parameters)
print section_query

<Wrap for http://api.brain-map.org/api/v2/data/SectionDataSet/query.json>


### Did you catch what Tortilla did there with the "SectionDataSet" attribute?

Calling `allen.SectionDataSet` added `/SectionDataSet` to the url defined in `allen`. If your API is hierarchical, you can traverse the whole thing like this. Slick, huh?

Then, the `query.json` argument passed into that gets appended as `/query.json` as well.

Nicely, this is just a wrapper, so it won't be executed until we call `.get()`, at which point the parameters will be appended as well.

In [4]:
response = section_query.get()
type(response)

tortilla.utils.Bunch

### This response is a tortilla "Bunch" 

... at the simplist level, it maintains the JSON of the response in a dictionary structure:

In [5]:
print response.keys()
print response['total_rows']
print len(response['msg'])

[u'msg', u'total_rows', u'success', u'num_rows', u'start_row', u'id']
3319
2000


However, since it is a "Bunch", we can also access the key values through attributes.

In [6]:
print response.total_rows
print len(response.msg)

3319
2000


We can also grab the next 2000 rows by calling `.get()` again, but updating the `start_row` parameter.

In [7]:
response = section_query.get(params={'start_row':2000})
print response.start_row
print len(response.msg)

2000
1319


### We want all the rows, however

So, to make our life a bit easier, we'll define a generator so we can just loop through all of queries needed to assemble the full output of the query.

In [8]:
def wrapped_query(wrapped,num_rows):
    start_row = 0
    while True:
        
        # let's get the response and spit out the rows
        response = wrapped.get(params={
            'start_row': start_row,
            'num_rows': num_rows,
        })
        yield response.msg
        
        # if we are at the end, let's bail. otherwise set up to grab the next set of rows
        if len(response.msg) < num_rows:
            break
        else:
            start_row += len(response.msg)

### Generators are awesome

They are basically functions that you can iterate over.

On each iteration of this generator, it is going to query the API and spit out the rows. And each time it is called, it will increment `start_row` by the number of rows it just returned.

In [9]:
rows = []
for r in wrapped_query(section_query,1000):
    print len(r)
    rows += r
print rows[0]

1000
1000
1000
319
{u'weight': 5270, u'qc_date': u'2009-05-02T22:58:26Z', u'reference_space_id': 9, u'failed_facet': 734881840, u'rnaseq_design_id': None, u'storage_directory': u'/external/aibssan/production32/prod336/image_series_71670698/', u'id': 71670698, u'plane_of_section_id': 1, u'name': None, u'sphinx_id': 141427, u'blue_channel': None, u'green_channel': None, u'failed': False, u'delegate': False, u'specimen_id': 70896862, u'red_channel': None, u'expression': True, u'section_thickness': 25.0}
